In [ ]:
import import_ipynb
import structure

In [ ]:
class Greenhouse:
    def __init__(self, 
        time_period: s,
        target_DLI: mol_per_m2_day,
        target_temp: [C, C],
        target_humidity: [RH, RH],
        photoperiod: h,
    ):
        self.time_period: s = time_period
        self.target_DLI: mol_per_m2_day = target_DLI
        self.target_temp: C = target_temp
        self.target_humidity = target_humidity
        self.photoperiod: h = photoperiod

        # Init subsystems
        self.structure = Structure(structure_type="transparent")
        self.light = Light(barrel_count=self.structure.barrel_count)
        self.dehumidifier = Dehumidifier()
        self.heatpump = HeatPump()
        self.solarpanel = SolarPanel(transparency=solar_panel_transparency, efficiency=0.22, photoperiod=photoperiod, target_DLI=target_DLI)
        self.windturbine = WindTurbine(turbine_count)

        # Init crop
        self.crop = SweetBasil(plants_per_barrel=self.structure.plants_per_barrel, barrel_count=self.structure.barrel_count, time_period=self.time_period)

        # Registers
        self.energy_in_battery: list[kWh] = [0]
        self.energy_from_grid: list[kWh] = [0]
        self.last_temp: C = None
        self.last_humidity: RH = None
        self.last_CO2: ppm = None

        # Targets
        self.temp_target = [18, 28]
        self.humidity_target = [40, 70]

        # Set control based on strategy
        if strategy == "CO2_constant":
            self.keep_CO2_constant = True


    def run(self, data, hour: h):
        self.hour = hour

        # Get initial values of time period
        self.start_temp: C = self.last_temp or data.temp
        self.start_humidity: RH = self.last_humidity or data.humidity
        self.start_CO2: RH = self.last_CO2 or ambient_CO2

        irradiance_inside, irradiance_on_solar_panels, transparency, projected_DLI = self.solarpanel.dynamic_solar_irradiance(data.solarradiation)

        # Calculate photosynthetic radiation
        PPFD: mol_per_m2_s = irradiance_to_PPFD(irradiance_inside)
        PAR_inside: mol_per_m2 = PPFD * self.time_period

        # Calculate energy used for lighting
        lighting_results = self.calculate_lighting(PAR_inside)

        # Calculate solar energy generation
        solar_energy_generated: kWh = self.solarpanel.calculate_solar_power(irradiance_on_solar_panels, self.structure.irradiated_area, self.time_period)

        # Calculate wind energy generation
        wind_power: W = self.windturbine.calculate_wind_power(data.wspd)
        wind_energy_generated: kWh = wind_power * self.time_period / 3.6e+6

        # Grow plants
        CO2_assimilated_mol, H2O_evaporated_mol = self.crop.grow(lighting_results["actual_PAR"] * 16 * (3600 / self.time_period), hour)

        (inside_temp, 
        inside_rel_humidity, 
        CO2_concentration, 
        airflow, 
        energy_for_dehumidification, 
        humidity_ratio, 
        energy_for_heating, 
        dehum_rate,
        CO2_supplemented) = self.register_airflow(
            H2O_evaporated_mol,
            data.humidity,
            data.temp,
            irradiance_inside,
            CO2_assimilated_mol
        )

        # Calculate energy used for dehumidification
        H2O_evaporated: l = (H2O_evaporated_mol * 18) / 1000
        energy_to_condense_all_water: kWh = self.dehumidifier.get_energy_usage(H2O_evaporated)

        # Calculate net energy
        total_energy_used: kWh = energy_for_dehumidification + lighting_results["energy_used_for_lighting_kWh"] + energy_for_heating
        total_energy_generated: kWh = solar_energy_generated + wind_energy_generated
        net_energy: kWh = total_energy_generated - total_energy_used

        # if net_energy > 0:
        #     # More energy was generated than used, store it in battery
        #     self.energy_in_battery.append(self.energy_in_battery[-1] + net_energy)
        # else:
        #     # More energy was used than generated
        #     if self.energy_in_battery[-1] > 0:
        #         # Use energy from battery
        #         if self.energy_in_battery[-1] > abs(net_energy):
        #             # All energy can be used from battery
        #             self.energy_in_battery.append(self.energy_in_battery[-1] - abs(net_energy))
        #         else:
        #             # Only some of the energy can be used from the battery
        #             energy_from_grid: kWh = abs(net_energy) - self.energy_in_battery[-1]
        #             self.energy_in_battery.append(0)
        #             self.energy_from_grid.append(energy_from_grid)
        #     else:
        #         # Use energy from grid
        #         self.energy_from_grid.append(abs(net_energy))

        return {
            "timestamp": data.timestamp,
            "baseline_CO2_supplemented": CO2_assimilated_mol * 1.2, # account for losses
            "baseline_energy_for_dehumidification": energy_to_condense_all_water,
            "total_energy_used_kWh": total_energy_used,
            "total_energy_generated_kWh": total_energy_generated,
            "net_energy_kWh": net_energy,
            "CO2_concentration": CO2_concentration,
            "outside_temp": data.temp,
            "outside_humidity": data.humidity,
            "inside_temp": inside_temp,
            "inside_rel_humidity": inside_rel_humidity,
            "humidity_ratio": humidity_ratio,
            "energy_for_dehumidification": energy_for_dehumidification,
            "dehum_rate": dehum_rate,
            "airflow": airflow,
            "CO2_supplemented": CO2_supplemented,
            "energy_for_heating": energy_for_heating,
            "transparency": transparency,
            "projected_DLI": projected_DLI,
            **lighting_results,
        }


    def calculate_lighting(self, natural_PAR: mol_per_m2):
        target_PAR_per_hour: mol_per_m2_hour = self.target_DLI / self.photoperiod
        hour_of_day = self.hour % 24

        target_PAR_current_hour: mol_per_m2_hour = target_PAR_per_hour
        if is_dark_hour(hour_of_day, self.photoperiod):
            target_PAR_current_hour = 0
        target_PAR: mol_per_m2 = target_PAR_current_hour * (self.time_period / 3600)
        target_PAR_total: mol = target_PAR * self.structure.barrel_surface_total
        natural_PAR_total: mol = natural_PAR * self.structure.barrel_surface_exposed_to_sun
        energy_used_for_lighting: kWh = 0
        supplemented_PAR_total: mol = 0
        wasted_PAR_total: mol_per_m2 = 0

        # Natural light is not enough, supplement needed
        if target_PAR_total > natural_PAR_total:
            supplemented_PAR_total = target_PAR_total - natural_PAR_total
            energy_used_for_lighting = self.light.get_energy_usage(supplemented_PAR_total, self.time_period)

        # Natural light is too much, PAR above target is wasted
        if target_PAR_total < natural_PAR_total:
            wasted_PAR_total = natural_PAR_total - target_PAR_total
        return {
            "energy_used_for_lighting_kWh": energy_used_for_lighting,
            "wasted_PAR": wasted_PAR_total / self.structure.barrel_surface_total,
            "actual_PAR": (natural_PAR_total + supplemented_PAR_total) / self.structure.barrel_surface_total,
        }

    def register_airflow(
            self, 
            H2O_evaporated: mol, 
            ambient_humidity: RH, 
            ambient_temp: C,
            irradiance_inside: W_per_m2,
            CO2_assimilated
        ):
            H2O_evaporated_per_s: mol_per_s = H2O_evaporated / self.time_period
            H2O_mass_evaporation_rate: g_per_s = H2O_evaporated_per_s * 18

            air_density: kg_per_m3 = 1.1839 # TODO: take humidity into account with psychrolib
            pressure: Pa = 101325
            air_mass_inside: kg = self.structure.volume * air_density

            CO2_assimilated_per_s: mol_per_s = CO2_assimilated / self.time_period

            def get_airflow(humidity_ratio: kg_H2O_per_kg_air, temp: C) -> m3_per_s:
                # Convert some humidity values to humidity ratios
                target_humidity_ratio: kg_H2O_per_kg_air = psychrolib.GetHumRatioFromRelHum(temp, self.target_humidity[1] / 100, pressure)

                # Default airflow
                airflow = 0.05

                # Proportionally increase airflow as humidity increases above target
                effective_target_humidity_ratio = target_humidity_ratio * 0.9
                if humidity_ratio > effective_target_humidity_ratio:
                    airflow = (humidity_ratio - target_humidity_ratio) * 20

                # Don't pump in air if outside is more humid than the target (at inside temp)
                ambient_humidity_ratio: kg_H2O_per_kg_air = psychrolib.GetHumRatioFromRelHum(ambient_temp, ambient_humidity / 100, pressure)
                ambient_humidity_at_inside_T = psychrolib.GetRelHumFromHumRatio(temp, ambient_humidity_ratio, pressure) * 100
                if ambient_humidity > (self.target_humidity[1] * 0.95):
                    airflow = 0

                if airflow > 1:
                    airflow = 1

                return airflow

            def derive_H2O(airflow: m3_per_s, humidity_ratio: kg_H2O_per_kg_air, temp: C, dehum_rate: g_per_s):
                humidity: RH = psychrolib.GetRelHumFromHumRatio(temp, humidity_ratio, pressure) * 100
                ambient_humidity_ratio: kg_H2O_per_kg_air = psychrolib.GetHumRatioFromRelHum(ambient_temp, ambient_humidity / 100, pressure)
                mass_airflow: kg_per_s = air_density * airflow

                if airflow == 0:
                    humidity_ratio_change_rate: g_per_s = H2O_mass_evaporation_rate - dehum_rate
                    dehum_rate_old: g_per_s = H2O_mass_evaporation_rate 
                else:
                    H2O_inflow: g_per_s = ambient_humidity_ratio * mass_airflow * 1000
                    H2O_outflow: g_per_s = humidity_ratio * mass_airflow * 1000
                    humidity_ratio_change_rate: g_per_s = H2O_inflow - H2O_outflow + H2O_mass_evaporation_rate - dehum_rate
                    dehum_rate_old: g_per_s = 0

                return humidity_ratio_change_rate, dehum_rate_old

            def derive_CO2(airflow, CO2_concentration: ppm):
                CO2_inflow: mol_per_s = ppm_to_amount(ambient_CO2, airflow)
                CO2_outflow: mol_per_s = ppm_to_amount(CO2_concentration, airflow)
                net_change_amount: mol_per_s = CO2_inflow - CO2_assimilated_per_s - CO2_outflow
                net_change_concentration: ppm_per_s = amount_to_ppm(net_change_amount, self.structure.volume)

                CO2_supplemented = 0
                if airflow == 0:
                    net_change_concentration = 0
                    CO2_supplemented: mol_per_s = CO2_assimilated_per_s

                return net_change_concentration, CO2_supplemented

            def derive_temp(airflow: m3_per_s, humidity_ratio: kg_H2O_per_kg_air, temp: C, dehum_rate: g_per_s, t) -> J_per_s:
                humidity: RH = psychrolib.GetRelHumFromHumRatio(temp, humidity_ratio, pressure) * 100

                if humidity > 100:
                    # print("Condensing!", new_humidity)
                    humidity = 100

                air_mass_flow: kg_per_s = air_density * airflow

                # Ambient air
                enthalpy_of_airflow_in: J_per_s = get_humid_air_specific_enthalpy(ambient_temp, ambient_humidity) * air_mass_flow

                # Inside air
                enthalpy_of_airflow_out: J_per_s = get_humid_air_specific_enthalpy(temp, humidity) * air_mass_flow

                # Sunlight
                power_irradiated: W = irradiance_inside * self.structure.irradiated_area
                radiation_loss = 0.2
                equipment_absorption_factor = 0.2
                enthalpy_absorbed_by_plants_and_air: J_per_s = power_irradiated * (1 - radiation_loss - equipment_absorption_factor)
                enthalpy_absorbed_by_evapotranspiration: J_per_s = H2O_mass_evaporation_rate * water_evaporation_heat
                enthalpy_absorbed_by_air: J_per_s = enthalpy_absorbed_by_plants_and_air - enthalpy_absorbed_by_evapotranspiration

                # Conductive loss
                conductive_enthalpy_loss: J_per_s = self.structure.get_heat_transfer_rate(temp - ambient_temp)

                # Condensation heat in dehumidifier
                condensation_heating_rate: J_per_s = water_evaporation_heat * dehum_rate

                # Net enthalpy change of air
                enthalpy_change_rate: J_per_s = enthalpy_of_airflow_in - enthalpy_of_airflow_out + enthalpy_absorbed_by_air - conductive_enthalpy_loss + condensation_heating_rate

                new_heating_rate: J_per_s = 0
                if airflow == 0 and enthalpy_change_rate > 0 and temp >= self.target_temp[1]:
                    new_heating_rate: J_per_s = enthalpy_change_rate
                    enthalpy_change_rate: J_per_s = 0

                return enthalpy_change_rate, new_heating_rate

            def model(y, t, dehum_rate):
                # Destructure y values
                humidity_ratio, spec_enthalpy, CO2, airflow, dehum_rate, heating_rate, CO2_supplemented = y

                # Convert some of them to different units
                temp: C = psychrolib.GetTDryBulbFromEnthalpyAndHumRatio(spec_enthalpy, humidity_ratio)
                temp = override_temp(temp)

                # Determine airflow
                new_airflow: m3_per_s = get_airflow(humidity_ratio, temp)

                # Calculate humidity change
                humidity_ratio_change_rate, new_dehum_rate = derive_H2O(new_airflow, humidity_ratio, temp, dehum_rate)
                enthalpy_change_rate, new_heating_rate = derive_temp(new_airflow, humidity_ratio, temp, new_dehum_rate, t)
                CO2_concentration_change, new_CO2_supplemented = derive_CO2(new_airflow, CO2)

                return [
                    humidity_ratio_change_rate, 
                    enthalpy_change_rate, 
                    CO2_concentration_change, 
                    new_airflow - airflow, 
                    new_dehum_rate - dehum_rate, 
                    new_heating_rate - heating_rate,
                    new_CO2_supplemented - CO2_supplemented
                ]

            # Set up initial values for odeint
            if self.start_humidity > 100:
                self.start_humidity = 100
            initial_humidity_ratio: kg_H2O_per_kg_air = psychrolib.GetHumRatioFromRelHum(self.start_temp, self.start_humidity / 100, pressure)
            initial_spec_enthalpy: J_per_kg = get_humid_air_specific_enthalpy(self.start_temp, self.start_humidity)
            initial_CO2: ppm = self.start_CO2
            initial_airflow: m3_per_s = 0
            initial_dehum_rate: g_per_s = 0
            initial_heating_rate: J_per_s = 0
            initial_CO2_supplemented: mol_per_s = 0
            initial_values = [
                initial_humidity_ratio, 
                initial_spec_enthalpy, 
                initial_CO2, 
                initial_airflow, 
                initial_dehum_rate, 
                initial_heating_rate,
                initial_CO2_supplemented
            ]

            # Set up controls
            dehum_rate: g_per_s = 0
            if self.start_humidity > self.target_humidity[1]:
                dehum_rate = (self.start_humidity - self.target_humidity[1]) * 0.1
                    

            # Set up intermediate points
            intermediate_points_per_hour = 60
            intermediate_points = np.linspace(0, self.time_period, intermediate_points_per_hour + 1)

            # Solve differential equation system
            values_in_period = odeint(model, initial_values, intermediate_points, (dehum_rate,))

            # Retrieve values at the end of the time_period
            new_humidity_ratio, new_spec_enthalpy, new_CO2, new_airflow, new_dehum_rate, new_heating_rate, new_CO2_supplemented = values_in_period[-1]
            new_temp: C = psychrolib.GetTDryBulbFromEnthalpyAndHumRatio(new_spec_enthalpy, new_humidity_ratio)
            new_temp = override_temp(new_temp)
            new_rel_humidity = psychrolib.GetRelHumFromHumRatio(new_temp, new_humidity_ratio, pressure) * 100

            # Calculate energy used for dehumidification
            dehum_rates = np.array([val[4] for val in values_in_period])
            removed_water: g = dehum_rates * 3600 / intermediate_points_per_hour
            removed_water: l = removed_water.sum() / 1000
            energy_for_dehumidification: kWh = self.dehumidifier.get_energy_usage(removed_water)

            # Calculate energy used for heating
            heating_rates = np.array([val[5] for val in values_in_period])
            handled_heat: J = heating_rates * 3600 / intermediate_points_per_hour
            energy_for_heating: kWh = self.heatpump.get_energy_usage(handled_heat.sum())

            # Calculate total CO2 supplemented
            CO2_supplement_rates = np.array([val[6] for val in values_in_period])
            CO2_supplemented: mol = (CO2_supplement_rates * 3600 / intermediate_points_per_hour).sum()

            # Save values to be used as starting values in inext iteration
            self.last_temp = new_temp
            self.last_humidity = new_rel_humidity
            self.last_CO2 = new_CO2

            return [
                new_temp, 
                new_rel_humidity, 
                new_CO2, 
                new_airflow, 
                energy_for_dehumidification, 
                new_humidity_ratio, 
                energy_for_heating, 
                new_dehum_rate, 
                CO2_supplemented
            ]


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=2f5dc715-67f7-4c8c-98f7-a87b736d3338' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>